# The data

https://www.kaggle.com/datasets/mohamedlotfy50/wmt-2014-english-french/data
There over 4.5 million sentence pairs available. However, I will only use 25,000 pairs due to computational feasiblility.


In [1]:
import pandas as pd
import numpy as np

n_sentences = 25000

data = pd.read_csv(
    "./data/en-fr/wmt14_translate_fr-en_train.csv", nrows=n_sentences
).dropna()

data.head()

,en,fr
0,Resumption of the session,Reprise de la session
1,I declare resumed the session of the European ...,Je déclare reprise la session du Parlement eur...
2,"Although, as you will have seen, the dreaded '...","Comme vous avez pu le constater, le grand ""bog..."
3,You have requested a debate on this subject in...,Vous avez souhaité un débat à ce sujet dans le...
4,"In the meantime, I should like to observe a mi...","En attendant, je souhaiterais, comme un certai..."


# spliting the sentences into tokens


In [2]:
import random

original_en_sentences = [sent.strip().split(" ") for sent in data["en"]]
original_fr_sentences = [sent.strip().split(" ") for sent in data["fr"]]


for i in range(3):
    index = random.randint(0, 10000)
    print("English: ", " ".join(original_en_sentences[index]))
    print("French: ", " ".join(original_fr_sentences[index]), "\n")

English:  (The President cut the speaker off)
French:  (La Présidente retire la parole à l' orateur) 

English:  We can deal with it.
French:  Nous pouvons le prendre en main. 

English:  Under the system proposed in these pieces of legislation, third country nationals wishing to move to the UK under the procedures described, will do so by virtue of the service provision card issued by another Member State, thereby by-passing UK border controls.
French:  Aux termes du système proposé dans ces dispositions législatives, les ressortissants de pays tiers désireux de se rendre au Royaume-Uni selon les procédures décrites le feront en vertu de la carte de prestation de services délivrée par un autre État membre, en contournant donc les contrôles frontaliers britanniques. 



# Adding special tokens

#### I will add "< s >" to mark the start of a sentence and "< /s >" to mark the end of a sentence

This way
we prediction can be done for an arbitrary number of time steps. Using < s > as the starting token gives a
way to signal to the decoder that it should start predicting tokens from the target language.

if < /s > token is not used to mark the end of a sentence, the decoder cannot be signaled to
end a sentence. This can lead the model to enter an infinite loop of predictions.


In [3]:
en_sentences = [["<s>"] + sent + ["</s>"] for sent in original_en_sentences]
fr_sentences = [["<s>"] + sent + ["</s>"] for sent in original_fr_sentences]

for i in range(2):
    index = random.randint(0, 10000)
    print("English: ", " ".join(en_sentences[index]))
    print("German: ", " ".join(fr_sentences[index]), "\n")

English:  <s> By having the opportunity to show their true merit, by eliminating as far as possible the barriers associated to the specifics of their condition as women, and not in the context of a conflict in which women seem to be attacking male privilege, women will demonstrate the benefits to be gained by facilitating a situation where their professional careers may flourish. In this way they will succeed in altering balances which are still not in their favour. </s>
German:  <s> Car c'est en ayant l'occasion de démontrer leurs réels mérites, en supprimant au maximum les barrières attachées à la spécificité de leur condition, et non dans le cadre d'un conflit où elles donneront l'impression de s'attaquer aux privilèges des hommes, que les femmes feront la preuve de l'intérêt de faciliter l'éclosion de leurs carrières professionnelles, et arriveront à la modification d'équilibres qui leur sont encore trop défavorables. </s> 

English:  <s> What it comes down to is the competitivenes

# splitting training and validation dataset

#### 80% training, 10% validation and 10% for testing


In [4]:
from sklearn.model_selection import train_test_split
import numpy as np

(
    train_en_sentences,
    valid_test_en_sentences,
    train_fr_sentences,
    valid_test_fr_sentences,
) = train_test_split(en_sentences, fr_sentences, test_size=0.2)


(valid_en_sentences, test_en_sentences, valid_fr_sentences, test_fr_sentences) = (
    train_test_split(valid_test_en_sentences, valid_test_fr_sentences, test_size=0.5)
)


print(train_en_sentences[1])
print(train_fr_sentences[1])
print("\n")
print(test_en_sentences[0])
print(test_fr_sentences[0])

['<s>', 'We', 'had', 'a', 'White', 'Paper', 'on', 'innovation', 'some', 'years', 'ago', '-', 'what', 'has', 'happened', 'since?', '</s>']
['<s>', 'Nous', 'avons', 'eu', 'un', 'Livre', 'blanc', 'sur', "l'innovation", 'il', 'y', 'a', 'quelques', 'années', '-', 'que', "s'est-il", 'passé', 'depuis', '?', '</s>']


['<s>', 'But,', 'when', 'all', 'is', 'said', 'and', 'done,', 'this', 'summit', 'will', 'have', 'a', 'major', 'benefit', '-', 'that', 'of', 'outlining,', 'in', 'the', 'wake', 'of', 'Cardiff', 'and', 'Luxembourg,', 'a', 'political', 'desire', 'for', 'greater', 'social', 'cohesion,', 'particularly', 'as', 'regards', 'the', '57', 'million', 'people', 'who', 'are', 'living', 'in', 'a', 'state', 'of', 'poverty', 'within', 'the', 'Union,', 'including', 'single', 'mothers,', 'large', 'families', 'and', 'children,', 'and', 'whose', 'numbers', 'will', 'in', 'the', 'future', 'be', 'swelled', 'by', 'other', 'outcasts', 'from', 'society,', 'specifically', 'those', 'people', 'who', 'will', 'no

### Defining sequence leghts fot the two languages


In [5]:
# Getting some basic statistics from the data

# convert train_en_sentences to a pandas series
pd.Series(train_en_sentences).str.len().describe(percentiles=[0.05, 0.5, 0.95])

count    20000.000000
mean        27.600050
std         15.726095
min          3.000000
5%           8.000000
50%         25.000000
95%         58.000000
max        150.000000
dtype: float64

In [6]:
pd.Series(train_fr_sentences).str.len().describe(percentiles=[0.05, 0.5, 0.95])

count    20000.000000
mean        28.900500
std         16.733816
min          3.000000
5%           8.000000
50%         26.000000
95%         61.000000
max        154.000000
dtype: float64

# from the train data statistics above, 95% of the english sentences have lengths of 57 while in the french, 95 % of sentences have lengths of 60


### padding the sentences with pad_sequences from keras


In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

n_en_seq_length = 60
n_de_seq_length = 60
unk_token = "<unk>"

train_en_sentences_padded = pad_sequences(
    train_en_sentences,
    maxlen=n_en_seq_length,
    # value=unk_token,
    dtype=object,
    truncating="post",
    padding="post",
)

valid_en_sentences_padded = pad_sequences(
    valid_en_sentences,
    maxlen=n_en_seq_length,
    # value=unk_token,
    dtype=object,
    truncating="post",
    padding="post",
)

test_en_sentences_padded = pad_sequences(
    test_en_sentences,
    maxlen=n_en_seq_length,
    # value=unk_token,
    dtype=object,
    truncating="post",
    padding="post",
)


train_fr_sentences_padded = pad_sequences(
    train_fr_sentences,
    maxlen=n_de_seq_length,
    # value=unk_token,
    dtype=object,
    truncating="post",
    padding="post",
)

valid_fr_sentences_padded = pad_sequences(
    valid_fr_sentences,
    maxlen=n_de_seq_length,
    # value=unk_token,
    dtype=object,
    truncating="post",
    padding="post",
)

test_fr_sentences_padded = pad_sequences(
    test_fr_sentences,
    maxlen=n_de_seq_length,
    # value=unk_token,
    dtype=object,
    truncating="post",
    padding="post",
)

print(train_en_sentences_padded[1])

2024-10-14 17:04:19.930947: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-14 17:04:19.939718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 17:04:19.949453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 17:04:19.952868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 17:04:19.960569: I tensorflow/core/platform/cpu_feature_guar

['<s>' 'We' 'had' 'a' 'White' 'Paper' 'on' 'innovation' 'some' 'years'
 'ago' '-' 'what' 'has' 'happened' 'since?' '</s>' 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0]


# Converting to token IDs


In [8]:
from tensorflow.keras.layers import TextVectorization
import os

# using text vectorization
# text_vectorizer_en = TextVectorization(output_mode="int")
# text_vectorizer_fr = TextVectorization(output_mode="int")
# text_vectorizer_en.adapt(data["en"])
# text_vectorizer_fr.adapt(data["fr"])

en_vocabulary = []
with open(os.path.join("./data/en-fr", "vocab.en"), "r", encoding="utf-8") as en_file:
    for ri, row in enumerate(en_file):

        en_vocabulary.append(row.strip())

fr_vocabulary = []
with open(os.path.join("./data/en-fr", "vocab.fr"), "r", encoding="utf-8") as en_file:
    for ri, row in enumerate(en_file):

        fr_vocabulary.append(row.strip())

text_vectorizer_en = TextVectorization(output_mode="int")
text_vectorizer_fr = TextVectorization(output_mode="int")
text_vectorizer_en.adapt(en_vocabulary)
text_vectorizer_fr.adapt(fr_vocabulary)


en_vocabulary = text_vectorizer_en.get_vocabulary()
fr_vocabulary = text_vectorizer_fr.get_vocabulary()

I0000 00:00:1728939862.001864   22433 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728939862.025891   22433 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728939862.025936   22433 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728939862.028989   22433 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728939862.029049   22433 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [9]:
en_unk_token = en_vocabulary.pop(1)
fr_unk_token = fr_vocabulary.pop(1)

en_unk_token, fr_unk_token

('[UNK]', '[UNK]')

In [12]:
import tensorflow as tf

pad_token = "[PAD]"

# English look up layer
en_lookup_layer = tf.keras.layers.StringLookup(
    vocabulary=en_vocabulary,
    oov_token=en_unk_token,
    mask_token=pad_token,
    pad_to_max_tokens=False,
)

# French look up layer
fr_lookup_layer = tf.keras.layers.StringLookup(
    vocabulary=fr_vocabulary,
    oov_token=en_unk_token,
    mask_token=pad_token,
    pad_to_max_tokens=False,
)